In [1]:
from utils import load_results
from games import get_game
import matplotlib.pyplot as plt
import ipywidgets as wid
import numpy as np
import seaborn as sns
from itertools import product
from welfare_functions import nash, util
sns.set()


In [5]:
general_game = get_game("SimpleDistInfComms", prior_1_param=[0], prior_2_param=[0], a=2)
priors = [a for a, b in general_game.generate_test_priors()]

@wid.interact(prior_1=(0, 1, 0.1), prior_2=(0, 1, 0.1), a=[1,2,5,10], eval_prior=priors, equi_prior=priors, babble=True, coord=True, leader=True, comm=True, other=True)
def plot_all_res(prior_1=0.5, prior_2=0.5, a=2, eval_prior='p_5', equi_prior='p_5', babble=True, coord=True, leader=True, comm=True, other=True):
    using = [babble, coord, leader, comm, other]
    names = ["Babble", "Coord", "Leader", "Comm", "Other"]
    names = [names[n] for n in range(len(names)) if using[n]]


    game = get_game("SimpleDistInfComms", prior_1_param=[prior_1], prior_2_param=[prior_2], a=a)
    save_folder = "SimpleDistInfComms_p{}_{}_a{}_naive_0".format(int(prior_1*10), int(prior_2*10), a)
    p1s, p2s = load_results(save_folder, 'Pols_0')
    r1s, r2s = load_results(save_folder, eval_prior + '_Pfs_0')
    # xr1s, xr2s = load_results(save_folder, eval_prior + '_XPfs_0')

    equi = game.get_equilibria(p1s, p2s, prior=equi_prior)
    equi_inds = []
    for i in range(len(r1s)):
        j = 0
        for e in equi:
            if i in e:
                break
            j+=1
        equi_inds.append(j)
    r1s = [r for i, r in enumerate(r1s) if using[equi_inds[i]]]
    r2s = [r for i, r in enumerate(r2s) if using[equi_inds[i]]]

    sns.scatterplot(x=r1s, y=r2s, fc='none', ec='orange', linewidth=1.3, label="Jointly Trained")
    # sns.scatterplot(x=xr1s, y=xr2s, fc='none', ec='blue', linewidth=1.3, label="Cross Tests")
    polygon = game.outcomes_polygon()
    plt.fill(polygon[0], polygon[1], alpha=0.1, color='purple')

interactive(children=(FloatSlider(value=0.5, description='prior_1', max=1.0), FloatSlider(value=0.5, descripti…